In [ ]:
%load_ext autoreload
%autoreload 2

import crispr as cr
from crispr.class_sc import Omics

file_path = {"directory": "data/crispr-screening/HH06"}
col_sample_id = "orig.ident"
kws_harmony = {"plot_convergence": True,
               "random_state": 1618}
kws_umap = {"min_dist": 0.3}
kws_init = {"assay": None, "assay_protein": None, 
            "col_gene_symbols": "gene_symbols", 
            "col_cell_type": "majority_voting", 
            "col_sample_id": None,
            "col_condition": None, 
            "key_control": None, 
            "key_treatment": None}
kws_pp = {"kws_hvg": {"min_mean": 0.0125, "max_mean": 10, 
                      "min_disp": 0.5, "flavor": "cell_ranger"}, 
          "target_sum": 10000.0, "cell_filter_pmt": [0, 15], 
          "cell_filter_ngene": [200, None], "cell_filter_ncounts": [3, 40000], 
          "gene_filter_ncell": [3, None], "regress_out": None, 
          "kws_scale": "log", "kws_umap": kws_umap}
kws_cluster = {"method_cluster": "leiden", "kws_umap": kws_umap, 
               "kws_neighbors": None, 
               "kws_pca": {"n_comps": None, "use_highly_variable": True}, 
               "kws_cluster": {"resolution": 0.5}}

# Single-Cell Omics Object Initialization

In [ ]:
self.adata

In [ ]:
self = Omics(file_path, **kwargs_init)

# Preprocessing

Perform normalization, filtering, QC, etc.

In [ ]:
self.preprocess(**kws_pp)

# Clustering

Use this method to perform dimensionality reduction and clustering.

Optionally, by specifying `model_celltypist`, we can also detect cell types using `CellTypist`.

You can see what models are available with this code:
```
import celltypist
celltypist.models.models_description().set_index("model").description.apply(lambda x: print(x))
```

Or it will show you valid models automatically if you feed it an unknown model: `self.cluster(**kws_cluster, model="kate_moss.pkl")``

You can perform this after clustering by leaving `model_celltypist` as None and at any point running `self.annotate_clusters(...)`.

In [ ]:
self.cluster(**kws_cluster, 
             model_celltypist="COVID19_HumanChallenge_Blood.pkl")

# Plots

Create a variety of plots visualizing gene expression, cell clusters, etc.

In [ ]:
clus = list(self.rna.obs["predicted_labels"].sample(2))  # clusters to circle
figs = self.plot(genes=16,  # or could specify list of gene names
                 col_cell_type="predicted_labels",
                 cell_types_circle=clus,  # list cell types to circle on UMAP
                 kws_gex_violin=dict(scale="area", height=10),
                 genes_highlight=None,  # could be genes names to highlight
                 kws_ump=dict(col_cell_type="predicted_labels"))

# Analyses

In [ ]:
cct = "majority_voting" if "majority_voting" in self.rna.obs else \
    self._columns["col_cell_type"]
# ccd = self._columns["col_perturbed"]
if self._columns["col_sample_id"] is None:
    self.rna.obs.loc[:, "tmp_sample_id_fake"] = ["1"] * int(
        self.rna.obs.shape[0] / 2) + ["2"] * (self.rna.obs.shape[0] - int(
            self.rna.obs.shape[0] / 2
            ))  # fake sample ID column for demonstration purposes
    csid = "tmp_sample_id_fake"
else:
    csid = self._columns["col_sample_id"]
oca = self.run_composition_analysis(est_fdr=0.05, 
                                    col_sample_id=csid,
                                    col_cell_type=cct,
                                    # col_condition=ccd,
                                    generate_sample_level=True)